# Using sqlalchemy and pandas to read and write from and to a local sqlite database for NLP

This post includes code adapted from [python for finance and trading algorithms udemy course](https://udemy.com/python-for-finance-and-trading-algorithms/) and [python for finance and trading algorithms udemy course notebooks](https://github.com/theoneandonlywoj/Python-for-Financial-Analysis-and-Algorithmic-Trading) and the [documentation here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.to_sql.html)

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('nf_complete.csv')

In [3]:
df.columns

Index(['Unnamed: 0', 'year', 'title', 'abstract', 'theme', 'China', 'Russia',
       'War', 'President', 'US', 'Vietnam', 'Cold War', 'World War',
       'Vietnam War', 'Korean War', 'Survey', 'Case Study', 'Trade',
       'Humanitarian', 'fixed_effects', 'instrumental_variable', 'regression',
       'experimental'],
      dtype='object')

In [4]:
df[["year","title"]]

,year,title
0,2000,"""Institutions at the Domestic/International Ne..."
1,2000,Born to Lose and Doomed to Survive: State Deat...
2,2000,The significance of “allegiance” in internatio...
3,2000,The significance of “allegiance” in internatio...
4,2000,Truth-Telling and Mythmaking in Post-Soviet Ru...
...,...,...
121,2018,Planning for the Short Haul: Trade Among Belli...
122,2018,Clinging to the Anti-Imperial Mantle: The Repu...
123,2018,The New Navy's Pacific Wars: Peripheral Confl...
124,2018,"Stop or I'll Shoot, Comply and I Won't: The Di..."


In [5]:
df_subset = df[["year", "title", "abstract", "theme", "War", 'Cold War',"Trade"]]
df_subset

,year,title,abstract,theme,War,Cold War,Trade
0,2000,"""Institutions at the Domestic/International Ne...",Civil-military relations are frequently studie...,IR scholarship,1,0,0
1,2000,Born to Lose and Doomed to Survive: State Deat...,"Under what conditions do states die, or exit t...",IR scholarship,1,1,0
2,2000,The significance of “allegiance” in internatio...,My dissertation employs original and secondary...,IR scholarship,1,0,0
3,2000,The significance of “allegiance” in internatio...,\nThis study revises prevailing interpretation...,Conflit Between States,0,1,0
4,2000,Truth-Telling and Mythmaking in Post-Soviet Ru...,Can distorted and pernicious ideas about histo...,Conflict Between States,1,0,0
...,...,...,...,...,...,...,...
121,2018,Planning for the Short Haul: Trade Among Belli...,"In times of war, why do belligerents continue ...",Conflict between states,1,0,1
122,2018,Clinging to the Anti-Imperial Mantle: The Repu...,"My dissertation project, Clinging to the Anti-...",Cold War,0,1,0
123,2018,The New Navy's Pacific Wars: Peripheral Confl...,Using a transnational methodology and sources ...,Military History,1,0,0
124,2018,"Stop or I'll Shoot, Comply and I Won't: The Di...",There is a dilemma at the heart of coercion. S...,IR Scholarship,0,0,1


In [6]:
df_subset.columns = ["year", "title", "abstract", "theme", "War", 'Cold War',"Trade"]

In [7]:
df_subset

,year,title,abstract,theme,War,Cold War,Trade
0,2000,"""Institutions at the Domestic/International Ne...",Civil-military relations are frequently studie...,IR scholarship,1,0,0
1,2000,Born to Lose and Doomed to Survive: State Deat...,"Under what conditions do states die, or exit t...",IR scholarship,1,1,0
2,2000,The significance of “allegiance” in internatio...,My dissertation employs original and secondary...,IR scholarship,1,0,0
3,2000,The significance of “allegiance” in internatio...,\nThis study revises prevailing interpretation...,Conflit Between States,0,1,0
4,2000,Truth-Telling and Mythmaking in Post-Soviet Ru...,Can distorted and pernicious ideas about histo...,Conflict Between States,1,0,0
...,...,...,...,...,...,...,...
121,2018,Planning for the Short Haul: Trade Among Belli...,"In times of war, why do belligerents continue ...",Conflict between states,1,0,1
122,2018,Clinging to the Anti-Imperial Mantle: The Repu...,"My dissertation project, Clinging to the Anti-...",Cold War,0,1,0
123,2018,The New Navy's Pacific Wars: Peripheral Confl...,Using a transnational methodology and sources ...,Military History,1,0,0
124,2018,"Stop or I'll Shoot, Comply and I Won't: The Di...",There is a dilemma at the heart of coercion. S...,IR Scholarship,0,0,1


In [8]:
import sqlalchemy as db
from sqlalchemy import create_engine
import sqlite3
import pandas as pd

In [9]:
engine = db.create_engine('sqlite:///nf_nlp.db')
connection = engine.connect()
metadata = db.MetaData()

In [10]:
nf_nlp_table = db.Table('nf_nlp_table', metadata, 
    db.Column('year',db.Integer, nullable=True, index=False),
    db.Column('title',db.String, nullable=True),
    db.Column('abstract',db.String, nullable=True),
    db.Column('theme',db.String, nullable=True),
    db.Column('War',db.Integer, nullable=True),
    db.Column('Cold War',db.Integer, nullable=True),
    db.Column('Trade', db.Integer, nullable=True)
)


In [11]:
metadata.create_all(engine) #Creates the table

In [12]:
nf_nlp_table

Table('nf_nlp_table', MetaData(bind=None), Column('year', Integer(), table=<nf_nlp_table>), Column('title', String(), table=<nf_nlp_table>), Column('abstract', String(), table=<nf_nlp_table>), Column('theme', String(), table=<nf_nlp_table>), Column('War', Integer(), table=<nf_nlp_table>), Column('Cold War', Integer(), table=<nf_nlp_table>), Column('Trade', Integer(), table=<nf_nlp_table>), schema=None)

In [13]:
df_subset.to_sql('nf_nlp_table', con=engine, if_exists='append', index=False)

In [16]:
engine.execute("SELECT year, theme, title FROM nf_nlp_table LIMIT 10").fetchall()

[(2000, 'IR scholarship', '"Institutions at the Domestic/International Nexus: the political-military  origins of military effectiveness, strategic integration and war'),
 (2000, 'IR scholarship', 'Born to Lose and Doomed to Survive: State Death and Survival in the International System'),
 (2000, 'IR scholarship', 'The significance of “allegiance” in international relations'),
 (2000, 'Conflit Between States', 'The significance of “allegiance” in international relations'),
 (2000, 'Conflict Between States', 'Truth-Telling and Mythmaking in Post-Soviet Russia: Historical Ideas, Mass Education, and Interstate Conflict'),
 (2000, 'Domestic Military History', 'Building a Cape Fear Metropolis: Fort Bragg, Fayetteville, and the  Sandhills of North Carolina'),
 (2000, 'Culture', 'The Glories and the Sadness: Shaping the national Memory of the First World War in Great Britain, Canada and Australia'),
 (2000, 'Culture / Peace Process', 'What leads longstanding adversaries to engage in conflict r

In [25]:
sql = """
SELECT
  year
, theme
, title
FROM nf_nlp_table
"""

cnxn = connection

In [26]:
df = pd.read_sql(sql, cnxn)

In [27]:
df.tail(30)

,year,theme,title
96,2014,IR Scholarship,“Multiparty Mediation: Identifying Characteris...
97,2014,IR Scholarship,The Justice Dilemma: International Criminal Ac...
98,2014,IR Scholarship,Beyond Revolution and Repression: U.S. Foreign...
99,2014,IR Scholarship,Protection States Trust?: Major Power Patronag...
100,2014,Nuclear Weapons,The Constraining Power of the Nuclear Nonproli...
101,2015,Military History,Selling Her the Military: Recruiting Women int...
102,2015,IR Scholarship,"American Evangelicals, Israel, and Modern Chri..."
103,2015,Non-state,Who Can Keep the Peace? Insurgent Organization...
104,2015,IR Scholarship,Credibility in Crisis: The Role of Leadership ...
105,2015,IR Scholarship,Evaluating the Changing of the Guards: Survey ...
